In [1]:
from src.data import DataLoaders
from src import utils
import pathlib
import pandas as pd
from src.models.ModelClasses import FireRiskModels, SmokeAlarmModels
from matplotlib import pyplot as plt

In [2]:
ACS =  DataLoaders.ACSData(2016,'block_group')

In [64]:
ACS.data

,inc_pct_poverty,race_pct_white,race_pct_whitenh,race_pct_nonwhite,race_pct_nonwhitenh,race_pct_amind,race_pct_black,race_pct_hisp,denom_age,age_under25_ma,...,house_yr_pct_before_1960,house_yr_pct_after_2000,house_yr_pct_1960_2000,house_val_less_50K,house_val_50_100K,house_val_100K_300K,house_val_300K_500K,house_val_more_500K,race_pct_black_or_amind,pct_alt_heat
GEOID,,,,,,,,,,,,,,,,,,,,,
010010201001,0.191946,0.785235,0.763758,0.214765,0.236242,0.000000,0.214765,0.021477,745.0,135.0,...,0.221831,0.193662,0.584507,0.152047,0.274854,0.508772,0.064327,0.000000,0.214765,0.017606
010010201002,0.044269,0.856126,0.826877,0.143874,0.173123,0.000000,0.082213,0.029249,1265.0,247.0,...,0.079229,0.085653,0.835118,0.100264,0.153034,0.538259,0.174142,0.034301,0.082213,0.032895
010010202001,0.382199,0.376042,0.376042,0.623958,0.623958,0.000000,0.591667,0.013542,960.0,283.0,...,0.135678,0.211055,0.653266,0.171053,0.337719,0.394737,0.065789,0.030702,0.591667,0.000000
010010202002,0.111426,0.497573,0.488673,0.502427,0.511327,0.000000,0.461974,0.012136,1236.0,221.0,...,0.235772,0.075203,0.689024,0.053691,0.513423,0.412752,0.020134,0.000000,0.461974,0.019912
010010203001,0.042301,0.626481,0.626481,0.373519,0.373519,0.009306,0.217851,0.096447,2364.0,399.0,...,0.030303,0.091887,0.877810,0.057522,0.349558,0.592920,0.000000,0.000000,0.227157,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560459511001,0.094029,0.950583,0.929307,0.049417,0.070693,0.000000,0.000000,0.021277,1457.0,154.0,...,0.308305,0.121729,0.569966,0.089320,0.114563,0.666019,0.091262,0.038835,0.000000,0.366569
560459511002,0.062579,0.935884,0.918642,0.064116,0.081358,0.004849,0.009698,0.018858,1856.0,250.0,...,0.197500,0.337500,0.465000,0.123438,0.079687,0.525000,0.232812,0.039062,0.014547,0.825737
560459513001,0.261512,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1151.0,189.0,...,0.556757,0.000000,0.443243,0.229282,0.303867,0.466851,0.000000,0.000000,0.000000,0.105163


In [3]:
NFIRS = DataLoaders.NFIRSData('block_group',ACS.tot_pop)
NFIRS_Sev = DataLoaders.NFIRSData('block_group',ACS.tot_pop,sev=True)

/home/kelsonss/anaconda3/envs/rcp2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [66]:
NFIRS.data

,fdid,inc_date,state,oth_inj,oth_death,prop_loss,cont_loss,tot_loss,severe_fire,min_loss,had_inj,had_death,year
geoid,,,,,,,,,,,,,
021100005001,11100,2017-10-02,AK,0,0,0,0,0,not_sev_fire,no_min_loss,no_inj,no_death,2017
021100005001,11100,2017-10-08,AK,0,0,0,0,0,not_sev_fire,no_min_loss,no_inj,no_death,2017
021100006002,11100,2017-10-16,AK,0,0,0,5000,5000,not_sev_fire,no_min_loss,no_inj,no_death,2017
021100002004,11100,2017-10-19,AK,0,0,0,0,0,not_sev_fire,no_min_loss,no_inj,no_death,2017
021100002003,11100,2017-01-06,AK,0,0,300,0,300,not_sev_fire,no_min_loss,no_inj,no_death,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...
560430002002,43507,2017-01-24,WY,0,0,2000,400,2400,not_sev_fire,no_min_loss,no_inj,no_death,2017
560430003022,43507,2017-06-27,WY,0,0,1000,1500,2500,not_sev_fire,no_min_loss,no_inj,no_death,2017
560430003011,43507,2017-06-28,WY,0,0,5000,1000,6000,not_sev_fire,no_min_loss,no_inj,no_death,2017


In [4]:
ACS_cols_to_use =[ 'did_not_work_past_12_mo',
               'house_pct_live_alone',
               'pct_alt_heat',
               'house_yr_pct_before_1960',
               'inc_pct_poverty',
               'race_pct_black_or_amind']

In [5]:
FSM =  FireRiskModels('severity')
FPM =  FireRiskModels('propensity')

In [6]:
FPM.train(NFIRS,ACS,ACS_cols_to_use)

/home/kelsonss/Documents/GitHub/rcp2/src/models/ModelClasses.py:165: RuntimeWarning: All-NaN axis encountered
  mx = np.nanmax(X, axis =1)
/home/kelsonss/anaconda3/envs/rcp2/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1115: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


Predicting 2017:
[[147006  44455]
 [  2487   8835]]
0.8595566016456833
              precision    recall  f1-score   support

       False       0.98      0.77      0.86    191461
        True       0.17      0.78      0.27     11322

    accuracy                           0.77    202783
   macro avg       0.57      0.77      0.57    202783
weighted avg       0.94      0.77      0.83    202783

7.995516328457793
Feature ranking:
1. Max (0.385028)
2. Median (0.154249)
3. pct_alt_heat (0.088281)
4. house_pct_live_alone (0.079518)
5. race_pct_black_or_amind (0.075907)
6. house_yr_pct_before_1960 (0.073211)
7. inc_pct_poverty (0.072736)
8. did_not_work_past_12_mo (0.071069)


In [7]:
FSM.train(NFIRS_Sev,ACS,ACS_cols_to_use)

/home/kelsonss/Documents/GitHub/rcp2/src/models/ModelClasses.py:165: RuntimeWarning: All-NaN axis encountered
  mx = np.nanmax(X, axis =1)
/home/kelsonss/anaconda3/envs/rcp2/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1115: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


Predicting 2017:
[[147895  51155]
 [  1032   2701]]
0.8185501491043077
              precision    recall  f1-score   support

       False       0.99      0.74      0.85    199050
        True       0.05      0.72      0.09      3733

    accuracy                           0.74    202783
   macro avg       0.52      0.73      0.47    202783
weighted avg       0.98      0.74      0.84    202783

8.888891214390483
Feature ranking:
1. Median (0.313726)
2. Max (0.212111)
3. did_not_work_past_12_mo (0.082153)
4. race_pct_black_or_amind (0.080522)
5. pct_alt_heat (0.079632)
6. house_yr_pct_before_1960 (0.078414)
7. inc_pct_poverty (0.077514)
8. house_pct_live_alone (0.075928)


In [8]:
ARC =  DataLoaders.ARCPData()

In [67]:
Smoke = SmokeAlarmModels(ARC.data,ACS.data)

In [68]:
Smoke.train()

In [91]:
out = NFIRS.fires.copy()
out.columns =  'fires_per_1K_pop_' + out.columns

In [92]:
sev_fires = NFIRS_Sev.fires.copy()

sev_fires.columns = 'severe_fires_per_1K_pop_' + sev_fires.columns
out = out.merge(sev_fires, how= 'left', left_index= True, right_index = True)

In [93]:
out['fire_propensity_risk_score'] = FPM.test_prediction_probs[:,1]
out['fire_severity_risk_score'] = FSM.test_prediction_probs[:,1]


In [94]:
# Smoke Alarm model munging 


# add number of surveys recorded at the block level
num_surveys_block = Smoke.models['Block_Group']['num_surveys'].copy()
num_surveys_block.index = num_surveys_block.index.str[2:]


out['recorded_home_visits'] = num_surveys_block


# add the statistical model outputs 
Smoke_predicts = Smoke.models['MultiLevel'][
    [ 'detectors_working_prc','detectors_working_CI','geography']].copy()
Smoke_predicts.index = Smoke_predicts.index.str[2:]
out = out.merge(Smoke_predicts,how = 'left', left_index = True, right_index = True)

In [95]:
out['smoke_detector_risk_score'] = 1- out['detectors_working_prc']/100  

In [96]:
out['CombinedRiskScore'] = (out['fire_severity_risk_score'] 
                          + out['fire_propensity_risk_score'] 
                          + out['smoke_detector_risk_score']
                            ) * 100       

In [97]:
out = out.merge( ACS.data[ACS_cols_to_use] , how = 'left', left_index = True, right_index = True)

In [98]:
out.index = '#_' + out.index
out.to_csv(utils.DATA['model-outputs'] / 'CombinedModels.csv')

In [ ]:
#model Summary File 
# model accuracy 
# key vars used 
# ACS_Columns
# which ACS Year used 
# etc 


In [59]:
out

,fires_per_1K_pop_2009,fires_per_1K_pop_2010,fires_per_1K_pop_2011,fires_per_1K_pop_2012,fires_per_1K_pop_2013,fires_per_1K_pop_2014,fires_per_1K_pop_2015,fires_per_1K_pop_2016,fires_per_1K_pop_2017,fire_propensity_risk_score,...,detectors_working_CI,geography,smoke_detector_risk_score,CombinedRiskScore,did_not_work_past_12_mo,house_pct_live_alone,pct_alt_heat,house_yr_pct_before_1960,inc_pct_poverty,race_pct_black_or_amind
geoid,,,,,,,,,,,,,,,,,,,,,
#_010010201001,1.342282,2.684564,NaN,NaN,NaN,NaN,NaN,1.342282,1.342282,0.622548,...,1.089020,state,0.6862,181.504963,0.355019,0.221831,0.017606,0.221831,0.191946,0.214765
#_010010201002,1.581028,0.790514,NaN,1.581028,0.790514,NaN,1.581028,0.790514,NaN,0.966579,...,1.089020,state,0.6862,247.683725,0.309021,0.127193,0.032895,0.079229,0.044269,0.082213
#_010010202001,1.047120,NaN,3.141361,NaN,1.047120,1.047120,1.047120,1.047120,NaN,0.740453,...,1.089020,state,0.6862,150.920717,0.509589,0.344560,0.000000,0.135678,0.382199,0.591667
#_010010202002,2.832861,2.832861,2.832861,0.944287,0.944287,NaN,0.944287,NaN,3.777148,0.853479,...,1.089020,state,0.6862,234.487162,0.309886,0.338496,0.019912,0.235772,0.111426,0.461974
#_010010203001,0.846024,1.269036,0.846024,0.423012,1.269036,0.423012,0.423012,0.846024,NaN,0.994841,...,1.089020,state,0.6862,267.734418,0.294054,0.276699,0.000000,0.030303,0.042301,0.227157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
#_560459511001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.372684,NaN,0.652114,...,3.199404,state,0.6110,215.638834,0.301663,0.324047,0.366569,0.308305,0.094029,0.000000
#_560459511002,0.632111,NaN,NaN,NaN,NaN,NaN,0.632111,NaN,NaN,0.592363,...,3.199404,state,0.6110,138.220686,0.288010,0.258713,0.825737,0.197500,0.062579,0.014547
#_560459513001,0.868810,NaN,0.868810,NaN,0.868810,0.868810,NaN,NaN,1.737619,0.918188,...,3.199404,state,0.6110,249.220401,0.232123,0.330784,0.105163,0.556757,0.261512,0.000000
